#### 实验介绍：
##### 将介绍如何通过 Amazon Bedrock 和 CrewAI 框架利用 Anthropic 的 Claude-3 语言模型的强大功能，从而将大型语言模型和代理工作流的定义无缝集成到自定义 AI 应用程序中

In [1]:
pip install -Uq langchain-anthropic crewai langchain-aws boto3 langchain_community  crewai_tools

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_aws import ChatBedrock
from langchain_core.messages import HumanMessage

In [3]:
chat = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs={"temperature": 0.1},
)

In [4]:
messages = [
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    )
]
chat.invoke(messages)

AIMessage(content="Voici la traduction en français :\n\nJ'aime la programmation.", additional_kwargs={'usage': {'prompt_tokens': 20, 'completion_tokens': 21, 'total_tokens': 41}}, response_metadata={'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0', 'usage': {'prompt_tokens': 20, 'completion_tokens': 21, 'total_tokens': 41}}, id='run-04c63f31-9039-4937-8b0e-6256f1f1042e-0')

##### 流式输出

In [5]:
for chunk in chat.stream(messages):
    print(chunk.content, end="", flush=True)

Voici la traduction en français :

J'aime la programmation.

##### 使用ChatPromptTemplate

In [6]:
from langchain_core.prompts import ChatPromptTemplate

In [7]:
system = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Chinese",
        "text": "I love Python",
    }
)

AIMessage(content='我喜欢Python', additional_kwargs={'usage': {'prompt_tokens': 22, 'completion_tokens': 10, 'total_tokens': 32}}, response_metadata={'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0', 'usage': {'prompt_tokens': 22, 'completion_tokens': 10, 'total_tokens': 32}}, id='run-5c741b00-3b68-47fd-8a7c-353a9fdaa947-0')

##### ChatAnthropic 也支持async和streaming功能:

In [8]:
prompt = ChatPromptTemplate.from_messages([("human", "Tell me a joke about {topic}")])
chain = prompt | chat
await chain.ainvoke({"topic": "bear"})

AIMessage(content="Here's a bear joke for you:\n\nWhy did the bear dissolve in water?\nBecause it was a polar bear!", additional_kwargs={'usage': {'prompt_tokens': 13, 'completion_tokens': 29, 'total_tokens': 42}}, response_metadata={'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0', 'usage': {'prompt_tokens': 13, 'completion_tokens': 29, 'total_tokens': 42}}, id='run-97b5c295-5616-4dfb-be2a-4501bf111fb1-0')

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [("human", "Give me a list of famous tourist attractions in Japan")]
)
chain = prompt | chat
for chunk in chain.stream({}):
    print(chunk.content, end="", flush=True)

Here are some of the most famous tourist attractions in Japan:

1. Mount Fuji - The iconic snow-capped mountain and highest peak in Japan.
2. Imperial Palace and Gardens (Tokyo) - The primary residence of the Emperor of Japan.
3. Sensoji Temple (Tokyo) - An ancient Buddhist temple and one of Tokyo's most colorful and popular temples.
4. Fushimi Inari Shrine (Kyoto) - Known for its thousands of vermilion torii gates.
5. Hiroshima Peace Memorial Park and Museum - A memorial to the atomic bombing of Hiroshima in 1945.
6. Kinkakuji (Golden Pavilion) (Kyoto) - A Zen Buddhist temple with a golden pavilion set in a beautiful garden.
7. Himeji Castle - One of Japan's most impressive and well-preserved feudal castles.
8. Miyajima Island and Itsukushima Shrine - A UNESCO World Heritage site with a famous "floating" torii gate.
9. Shibuya Crossing (Tokyo) - One of the busiest pedestrian crossings in the world.
10. Jigoku-Meguri (Hell Tour) (Beppu) - A tour of hot springs and steaming pools resemb

###### 接下来将建立了一个名为“ 资深行业分析师“  的代理，旨在发现人工智能和数据科学的前沿发展。然后，它分配了一项任务，对 2024 年最新的人工智能进展进行全面分析，预期输出是一份要点报告。最后，它使用代理和任务创建一个“Crew”实例，并执行工作流

In [10]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool

researcher = Agent(
 role='资深行业分析师',
 goal='揭示人工智能和数据科学的前沿发展',
 backstory="你在一家领先的科技智库工作…",
 verbose=True,
 allow_delegation=False,
 llm=chat,
 tool=[SerperDevTool()]
)
task1 = Task(
 description="对2024年人工智能的最新进展进行全面分析…",
 expected_output="中文分析报告",
 agent=researcher
)
crew = Crew(
 agents=[researcher],
 tasks=[task1],
 verbose=2,
)

In [11]:
result = crew.kickoff()

 [DEBUG]: == Working Agent: 资深行业分析师
 [INFO]: == Starting Task: 对2024年人工智能的最新进展进行全面分析…


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer:

2024年人工智能最新进展全面分析报告

前言

人工智能(AI)是当今科技领域最具革命性和颠覆性的技术之一。近年来,AI的发展一日千里,给各行各业带来了深刻的影响和变革。2024年,AI的创新步伐将进一步加快,预计将在多个领域取得突破性进展。本报告将全面分析2024年人工智能的最新发展趋势和热点领域。

大模型与通用人工智能

2024年,大模型和通用人工智能(AGI)将成为人工智能领域的核心研究方向。大模型通过海量数据训练,展现出惊人的泛化能力,可以应用于多种任务。2024年,大模型的规模和性能将进一步提升,有望在自然语言处理、计算机视觉、决策推理等多个领域取得突破。

与此同时,通用人工智能的研究也将加速推进。科学家们将致力于开发具有类人般智能的AGI系统,以期实现真正的人工通用智能。虽然AGI的实现仍面临重重挑战,但2024年有望在理论建模、算法优化等方面取得重大进展。

人工智能与元宇宙融合

元宇宙被视为未来数字世界的重要组成部分,而人工智能则是实现元宇宙的关键技术支撑。2024年,人工智能与元宇宙的融合将成为一大热点。AI技术将广泛应用于元宇宙中的虚拟人物建模、场景生成、交互体验优化等领域,为用户带来更加身临其境的沉浸式体验。

同时,元宇宙也将为人工智能提供海量的真实数据和应用场景,推动AI算法和模型的进一步优化和发展。人工智能与元宇宙的深度融合,将催生出新的应用场景和商业模式。

人工智能伦理与隐私保护

随着人工智能的不断发展,AI伦理和隐私保护问题日益受到重视。2024年,这一领域将成为人工智能研究的重点。科学家们将致力于建立AI伦理准则和隐私保护机制,确保人工智能系统的公平性、透明度和可解释性,避免潜在的歧视和隐私泄露风险。

同时,相关法律法规的制定和完善也将成为重中之重,以规范人工智能的研发和应用,保护公众利益。人工智能伦理和隐私保护将成为人工智能可持续发展的基石。

结语
